In [2]:
import os
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
from dash import dash_table
from dash.dependencies import Input, Output, State
import pandas as pd
import plotly.express as px
import base64
import logging

from AnimalShelter import AnimalShelter

###########################
# Configuration
###########################

# Load configuration from environment variables
username = os.environ.get("USERNAME")
password = os.environ.get("PASSWORD")
auth_source = os.environ.get("AUTH_SOURCE")
image_filename = os.environ.get("IMAGE_FILENAME")

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

###########################
# Data Access
###########################

def fetch_animal_data():
    shelter = AnimalShelter(username, password, auth_source)
    return pd.DataFrame.from_records(shelter.findDocs({})).drop('_id', axis=1)

###########################
# Dashboard Layout / View
###########################

def create_logo(image_filename):
    try:
        with open(image_filename, 'rb') as f:
            encoded_image = base64.b64encode(f.read()).decode()
            return html.Center(html.Img(src="data:image/png;base64,{}".format(encoded_image)))
    except FileNotFoundError:
        logger.warning("Logo image file not found")
        return ""

def create_table(df):
    return dash_table.DataTable(
        id='datatable-interactivity',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action='custom',
        sort_mode="multi",
        sort_by=[],
        row_selectable="single",
        row_deletable=True,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10
    )

def create_dashboard_layout(image_filename, df):
    return html.Div([
        html.Center(html.B(html.H1('Brandon Petersen', style={'color': 'green'}))),
        html.Center(html.B(html.H1('SNHU CS340: Client/Server Development', style={'color': 'blue'}))),
        create_logo(image_filename),
        html.Hr(),
        html.Div([
            html.Center(html.Label(['Select an option to filter the table by rescue type:'], style={'font-weight': 'bold'})),
            html.Center(dcc.RadioItems(
                id='radio-items',
                options=[
                    {'label': 'Water Rescue', 'value': 'Filter by Water Rescue'},
                    {'label': 'Mountain Rescue', 'value': 'Filter by Mountain Rescue'},
                    {'label': 'Disaster Rescue', 'value': 'Filter by Disaster Rescue'},
                    {'label': 'Reset', 'value': 'Remove all filters'}
                ],
                value='Filter table by rescue dog types',
                inputStyle={"margin-left": "20px"},
                labelStyle={'display': 'inline-block'}
            ))
        ]),
        html.Br(),
        create_table(df),
        html.Br(),
        html.Hr(),
        html.Div(className='row', style={'display': 'flex', "width": "500"}, children=[
            html.Div(className='row', style={'display': 'flex'}, children=[
                dcc.Graph(
                    id='pie-chart',
                    style={'width': '50%', 'display': 'inline-block'}
                )
            ]),
            html.Div(className='row', style={'display': 'flex'}, children=[
                dl.Map(
                    id='geolocation-plot',
                    style={'width': '1000px', 'height': '500px', 'margin': 'auto', 'display': 'block'},
                    center=[df.iloc[0]['location_lat'], df.iloc[0]['location_long']],
                    zoom=10,
                    children=[
                        dl.TileLayer(id="base-layer-id"),
                        dl.Marker(
                            position=[df.iloc[0]['location_lat'], df.iloc[0]['location_long']],
                            children=[
                                dl.Tooltip(df.iloc[0]['breed'])
                            ]
                        )
                    ]
                )
            ])
        ])
    ])

###########################
# Interaction Between Components / Controller
###########################

def filter_table(df, rescue_type):
    if rescue_type == 'Remove all filters':
        return df.to_dict('records')
    else:
        filtered_df = df[df['breed'] == rescue_type]
        return filtered_df.to_dict('records')

def update_pie_chart(df, rows, selected_rows):
    if selected_rows:
        selected_data = pd.DataFrame(rows).loc[selected_rows]
        fig = px.pie(selected_data, names='breed')
        return fig
    else:
        fig = px.pie(df, names='breed')
        return fig

def update_geolocation_chart(df, rows, selected_rows):
    if selected_rows:
        selected_data = pd.DataFrame(rows).loc[selected_rows]
        children = [
            dl.TileLayer(id="base-layer-id"),
        ]

        for _, row in selected_data.iterrows():
            children.append(
                dl.Marker(
                    position=[row['location_lat'], row['location_long']],
                    children=[
                        dl.Tooltip(row['breed'])
                    ]
                )
            )

        return children
    else:
        children = [
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(
                position=[df.iloc[0]['location_lat'], df.iloc[0]['location_long']],
                children=[
                    dl.Tooltip(df.iloc[0]['breed'])
                ]
            )
        ]
        return children

###########################
# Dashboard App
###########################

def create_dashboard_app(df):
    app = dash.Dash(__name__, external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])

    app.layout = create_dashboard_layout(image_filename, df)

    @app.callback(
        Output('datatable-interactivity', 'data'),
        [Input('radio-items', 'value')]
    )
    def update_table(rescue_type):
        return filter_table(df, rescue_type)

    @app.callback(
        Output('pie-chart', 'figure'),
        [Input('datatable-interactivity', 'derived_virtual_data'),
         Input('datatable-interactivity', 'derived_virtual_selected_rows')]
    )
    def update_pie(rows, selected_rows):
        return update_pie_chart(df, rows, selected_rows)

    @app.callback(
        Output('geolocation-plot', 'children'),
        [Input('datatable-interactivity', 'derived_virtual_data'),
         Input('datatable-interactivity', 'derived_virtual_selected_rows')]
    )
    def update_geolocation(rows, selected_rows):
        return update_geolocation_chart(df, rows, selected_rows)

    return app

###########################
# Main
###########################

if __name__ == '__main__':
    # Fetch animal data
    df = fetch_animal_data()

    # Create dashboard app
    app = create_dashboard_app(df)

    # Run the app
    app.run_server(debug=True)


ModuleNotFoundError: No module named 'dash'